In [10]:
import os
os.chdir('../')
print(os.getcwd())
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from dataset.dataloader import Dataloader
from dataset.dataset import Dataset

In [6]:
data = Dataloader(
    listing_path = 'dataset/listings',
    comments_path = 'dataset/comments',
)
df = data.getListings()

scores_conlumns = [ col for col in df.columns if "score" in col ]
other_possible_scores_conlumns = ["price"]
scores = df[scores_conlumns]

# loads pre_processed data
with open(f'embeddings/listing_name_embeddings.pkl', 'rb') as f:
    listing_name = pd.read_pickle(f)
    
with open(f'embeddings/listing_description_embeddings.pkl', 'rb') as f:
    listing_description = pd.read_pickle(f)

with open(f'embeddings/listing_neighborhood_overview_embeddings.pkl', 'rb') as f:
    listing_neighborhood = pd.read_pickle(f)

with open(f'embeddings/listing_host_about_embeddings.pkl', 'rb') as f:
    listing_host_about = pd.read_pickle(f)

with open(f'embeddings/listing_processed_data.pkl', 'rb') as f:
    listings_numeric = pd.read_pickle(f)

with open(f'embeddings/comments_embeddings.pkl', 'rb') as f:
    comments = pd.read_pickle(f)

with open(f'embeddings/comments_mean_emb.pkl', 'rb') as f:
    comments_mean = pd.read_pickle(f)

In [7]:
print("listing_name:", listing_name.shape)
print("listing_description:", listing_description.shape)
print("listing_neighborhood:", listing_neighborhood.shape)
print("listing_host_about:", listing_host_about.shape)
print("listings_numeric:", listings_numeric.shape)
print("comments:", comments.shape)
print("comments_mean:", comments_mean.shape)

listing_name: (6998, 384)
listing_description: (6998, 384)
listing_neighborhood: (6998, 384)
listing_host_about: (6998, 384)
listings_numeric: (6998, 146)
comments: (6998, 384)
comments_mean: (384,)


In [11]:
# Generate train - Test Split this is done Once to be able to compare results
comments_pd = pd.DataFrame(comments, columns= [ f'comments_average_embeddings_{i}' for i in range(comments.shape[1])])
dataset_pd = pd.concat([listings_numeric, listing_name, listing_description, listing_neighborhood, listing_host_about, comments_pd], axis=1)
dataset_pd.shape

(6998, 2066)

In [15]:
# using the score as a referenc to understand which rows to drop, if no score is aviable 
dataset_with_no_null_scores_x = dataset_pd[df['review_scores_rating'].notna()]
dataset_with_no_null_scores_y = scores[df['review_scores_rating'].notna()]
print(dataset_with_no_null_scores_x.shape)
print(dataset_with_no_null_scores_y.shape)

(6316, 2066)
(6316, 7)


In [13]:
x_train, x_test, y_train, y_test = train_test_split(dataset_with_no_null_scores_x, dataset_with_no_null_scores_y, test_size=0.2, random_state=0)

In [14]:
dataset_train = Dataset(x_train, y_train)
dataset_test = Dataset(x_test, y_test)

In [42]:
with open(f'{ROOT_PATH}/train_dataset.pkl', 'wb') as f:
    pickle.dump(dataset_train, f)

with open(f'{ROOT_PATH}/test_dataset.pkl', 'wb') as f:
    pickle.dump(dataset_test, f)